### Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [2]:
tf.__version__

'2.13.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

Mounting Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/Btech/dataset_ dru.zip'

Archive:  /content/drive/MyDrive/Btech/dataset_ dru.zip
   creating: dataset_ dru/testingData/
   creating: dataset_ dru/testingData/D/
  inflating: dataset_ dru/testingData/D/108.jpg  
  inflating: dataset_ dru/testingData/D/109.jpg  
  inflating: dataset_ dru/testingData/D/11.jpg  
  inflating: dataset_ dru/testingData/D/117.jpg  
  inflating: dataset_ dru/testingData/D/118.jpg  
  inflating: dataset_ dru/testingData/D/121.jpg  
  inflating: dataset_ dru/testingData/D/124.jpg  
  inflating: dataset_ dru/testingData/D/128.jpg  
  inflating: dataset_ dru/testingData/D/131.jpg  
  inflating: dataset_ dru/testingData/D/133.jpg  
  inflating: dataset_ dru/testingData/D/134.jpg  
  inflating: dataset_ dru/testingData/D/135.jpg  
  inflating: dataset_ dru/testingData/D/138.jpg  
  inflating: dataset_ dru/testingData/D/145.jpg  
  inflating: dataset_ dru/testingData/D/147.jpg  
  inflating: dataset_ dru/testingData/D/148.jpg  
  inflating: dataset_ dru/testingData/D/149.jpg  
  inflating: da

Function to normalize image according to pre-trained mode.

In [5]:
def normalize_data(img):


    #Normalize for ResNet50
  return tf.keras.applications.resnet50.preprocess_input(img)

In [6]:
train_datagen = ImageDataGenerator(
    preprocessing_function= normalize_data,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [7]:
test_datagen = ImageDataGenerator(preprocessing_function= normalize_data)

### Creating the Training set

In [8]:
training_set = train_datagen.flow_from_directory('/content/dataset_ dru/trainingData',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1401 images belonging to 3 classes.


In [9]:
test_set = test_datagen.flow_from_directory('/content/dataset_ dru/testingData',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 465 images belonging to 3 classes.


### Part 2 - Building the CNN

In [23]:
tf.keras.backend.clear_session()
model = tf.keras.applications.ResNet50(include_top=False, #Do not include classification layer for imagenet
                                       input_shape=(128,128,3),
                                       weights='imagenet')

In [24]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [25]:
len(model.layers)

175

In [26]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

In [27]:
#Set last few layers to trainable
for layer in model.layers[173:]:
    layer.trainable = True

In [28]:
#get Output layer of Pre0trained model
x1 = model.output

#Global average pool to reduce number of features and Flatten the output
x2 = tf.keras.layers.GlobalAveragePooling2D()(x1)

In [29]:
# adding extra layers
x3 = tf.keras.layers.Dense(1024,activation='relu',kernel_initializer= "he_uniform")(x2)
x4 = tf.keras.layers.Dropout(0.4)(x3)
x5=  tf.keras.layers.Dense(512,activation='relu',kernel_initializer= "he_uniform")(x4)


In [30]:
#Add output layer
prediction = tf.keras.layers.Dense(3,activation='softmax')(x5)

## Building final Model for Classification

In [31]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [32]:
#Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
#How does our overall /model looks
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [34]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model.h5",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [35]:
final_model.fit(training_set,
                  epochs = 30, steps_per_epoch=1401//32,
                validation_steps=465//32,
                  validation_data = test_set,callbacks = [model_checkpoint_callback])

Epoch 1/30
43/43 [==============================] - ETA: 0s - loss: 0.4826 - accuracy: 0.8488
Epoch 1: val_accuracy improved from -inf to 0.98661, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


43/43 [==============================] - 15s 231ms/step - loss: 0.4826 - accuracy: 0.8488 - val_loss: 0.0354 - val_accuracy: 0.9866
Epoch 2/30
43/43 [==============================] - ETA: 0s - loss: 0.0548 - accuracy: 0.9803
Epoch 2: val_accuracy did not improve from 0.98661
43/43 [==============================] - 9s 215ms/step - loss: 0.0548 - accuracy: 0.9803 - val_loss: 0.0729 - val_accuracy: 0.9643
Epoch 3/30
43/43 [==============================] - ETA: 0s - loss: 0.0577 - accuracy: 0.9803
Epoch 3: val_accuracy improved from 0.98661 to 0.99554, saving model to model.h5
43/43 [==============================] - 9s 205ms/step - loss: 0.0577 - accuracy: 0.9803 - val_loss: 0.0147 - val_accuracy: 0.9955
Epoch 4/30
43/43 [==============================] - ETA: 0s - loss: 0.0305 - accuracy: 0.9912
Epoch 4: val_accuracy improved from 0.99554 to 0.99777, saving model to model.h5
43/43 [==============================] - 8s 188ms/step - loss: 0.0305 - accuracy: 0.9912 - val_loss: 0.0098 - v

In [36]:
model_json = final_model.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
final_model.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved


In [37]:
from google.colab import files
files.download('model_new.json')
files.download('model_new.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# END OF TRANSFER LEARNING